<a href="https://www.kaggle.com/code/ugorjiir/netherlandenergy0?scriptVersionId=111769043" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

**Energy consumption of the Netherlands**


The energy network of the Netherlands is managed by a few companies. Every year, these companies release on their websites a table with the energy consumption of the areas under their administration. The companies are

 Enexis, Liander, Stedin, Enduris, Westlandinfra, Rendo, Coteq

The data are anonymized by aggregating the Zipcodes so that every entry describes at least 10 connections.

This market is not competitive, meaning that the zones are assigned. This means that every year they roughly provide energy to the same zipcodes. Small changes can happen from year to year either for a change of management or for a different aggregation of zipcodes.

Content
Every file contains information about groups of zipcodes managed by one of the three companies for a specific year.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**IMPORTING LIBRARIES AND DATASET**

In [ ]:
# Importing Libraries
import glob
import os 
from IPython import display
import pandas as pd
import numpy as np
from scipy import stats
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
ELECTRICITY_PATH = '../input/dutch-energy/Electricity'
GAS_PATH = '../input/dutch-energy/Gas'

Elecfiles = glob.glob(os.path.join(ELECTRICITY_PATH , "*.csv"))
Gasfiles = glob.glob(os.path.join(GAS_PATH , "*.csv"))

Eleclist = []
Gaslist = []

print(len(Elecfiles))

In [ ]:
# To get the list of dataframes in our electricity directory
for filename in Elecfiles:
    df = pd.read_csv(filename)
    #to get th year from our file name
    file_name = os.path.basename(filename)
    x = file_name.split('.')
    year = x[0][-4:]
    df['year'] = year
    Eleclist.append(df)
    
# To get the list of dataframes in our electricity directory
for filename in Gasfiles:
    df = pd.read_csv(filename, index_col=None, header=0)
    #to get th year from our file name
    file_name = os.path.basename(filename)
    #split files by the '.'
    x = file_name.split('.')
    #collect the year variable by the last four values 
    year = x[0][-4:]
    df['year'] = year
    Gaslist.append(df)
    
print(len(Eleclist))
print(len(Gaslist))

In [ ]:
# Create our final dataframes containing all our data
Elec_df = pd.concat(Eleclist, ignore_index=True)
Gas_df = pd.concat(Gaslist, ignore_index=True)

del(Eleclist)
del(Gaslist)

Elec_df.head(), Gas_df.head()

**DATA CLEANING**

In [ ]:
Elec_df.info()

In [ ]:
Gas_df.info()

In [ ]:
print(Elec_df.shape)
print(Gas_df.shape)

In [ ]:
print(Elec_df.isnull().sum().sort_values(ascending=False))
print(Gas_df.isnull().sum().sort_values(ascending=False))

We can observe a number of columns in both dataframes with a large amount of missing values, especially the last three values in the dataframes. We will also be dropping some irrelevant or less important columns to keep our dataframe compact.

In [ ]:
Elec_df = Elec_df.drop(['ï»¿NETBEHEERDER', '%Defintieve aansl (NRM)', 'STANDAARDDEVIATIE', 'purchase_area', 'net_manager'], axis=1)
Gas_df = Gas_df.drop(['ï»¿NETBEHEERDER', '%Defintieve aansl (NRM)', 'STANDAARDDEVIATIE', 'purchase_area', 'net_manager'], axis=1)

print(Elec_df.isnull().sum().sort_values(ascending=False))
print(Gas_df.isnull().sum().sort_values(ascending=False))

In [ ]:
# Dropping the relatively smaller missing values in their respective columns
Elec_df = Elec_df.dropna(subset=['num_connections', 'perc_of_active_connections', 'delivery_perc'])
Gas_df = Gas_df.dropna(subset=['zipcode_from', 'num_connections'])

In [ ]:
print(Elec_df.isnull().sum().sort_values(ascending=False))
print(Gas_df.isnull().sum().sort_values(ascending=False))

In [ ]:
Elec_df.describe()

In [ ]:
# Elec_df[Elec_df['annual_consume']<12000].sort_values(by='annual_consume', ascending=False) 
Elec_df = Elec_df[Elec_df['annual_consume']<12000]
# Elec_df[Elec_df['annual_consume']==0]
Elec_df = Elec_df[Elec_df['annual_consume']>0]
sns.kdeplot(x=Elec_df['annual_consume'])

In [ ]:
# Elec_df[Elec_df['num_connections']>100].sort_values(by='annual_consume', ascending=False)
Elec_df = Elec_df[Elec_df['num_connections']<130]
sns.kdeplot(x=Elec_df['num_connections'])

In [ ]:
# Elec_df['type_of_connection'].value_counts()
Elec_df[(Elec_df['annual_consume_lowtarif_perc'].isna()) & (Elec_df['smartmeter_perc'].isna())]
# Elec_df[(Elec_df['annual_consume_lowtarif_perc']==0) & (Elec_df['smartmeter_perc']==0)]
# Elec_df[(Elec_df['annual_consume_lowtarif_perc']==0) & (Elec_df['smartmeter_perc']>0)]
# Elec_df[(Elec_df['annual_consume_lowtarif_perc']>0) & (Elec_df['smartmeter_perc']==0)]

In [ ]:
Elec_df['annual_consume_lowtarif_perc'] = Elec_df['annual_consume_lowtarif_perc'].fillna(0)
Elec_df['smartmeter_perc'] = Elec_df['smartmeter_perc'].fillna(0)

In [ ]:
print(Elec_df.isnull().sum().sort_values(ascending=False))

In [ ]:
print(Elec_df['type_conn_perc'].dtype)
# Elec_df['type_conn_perc'].astype(float) ###shows error
print(len(Elec_df['type_conn_perc'].unique()))
Elec_df['type_conn_perc'] = Elec_df['type_conn_perc'].str.replace(',', '.')
Elec_df['type_conn_perc'] = Elec_df['type_conn_perc'].astype(float)
print(Elec_df['type_conn_perc'].dtype)

In [ ]:
print(Elec_df['type_of_connection'].mode())
print(Elec_df['type_conn_perc'].median())
Elec_df['type_of_connection'] = Elec_df['type_of_connection'].fillna(Elec_df['type_of_connection'].mode()[0])
Elec_df['type_conn_perc'] = Elec_df['type_conn_perc'].fillna(Elec_df['type_conn_perc'].median())
print(Elec_df.isnull().sum().sort_values(ascending=False))

In [ ]:
# Convert to Datetime
Elec_df['year'] = pd.to_datetime(Elec_df.year)

In [ ]:
print(Gas_df.isnull().sum().sort_values(ascending=False))

In [ ]:
Gas_df.describe()

In [ ]:
#Replacing these columns with 0 using our filter
Gas_df[(Gas_df['perc_of_active_connections'].isna()) & (Gas_df['delivery_perc'].isna()) & 
       (Gas_df['annual_consume_lowtarif_perc'].isna()) & (Gas_df['smartmeter_perc'].isna())] = 0
print(Gas_df.isnull().sum().sort_values(ascending=False))

In [ ]:
Gas_df[(Gas_df['smartmeter_perc'].isna()) & (Gas_df['annual_consume_lowtarif_perc'].isna())]

In [ ]:
Gas_df[Gas_df['delivery_perc']==100].describe()

In [ ]:
a = Gas_df[(Gas_df['smartmeter_perc'].isna()) & (Gas_df['annual_consume_lowtarif_perc'].isna())]['annual_consume_lowtarif_perc'].index
b = Gas_df[(Gas_df['smartmeter_perc'].isna()) & (Gas_df['annual_consume_lowtarif_perc'].isna())]['smartmeter_perc'].index

Gas_df.loc[a,'annual_consume_lowtarif_perc'] = Gas_df[Gas_df['delivery_perc']==100].median()['delivery_perc']
Gas_df.loc[b,'smartmeter_perc'] = Gas_df[Gas_df['delivery_perc']==100].median()['smartmeter_perc']
print(Gas_df.isnull().sum().sort_values(ascending=False))

In [ ]:
Gas_df['type_of_connection'].dtype
print(len(Gas_df['type_conn_perc'].unique()))
Gas_df['type_conn_perc'] = Gas_df['type_conn_perc'].str.replace(',', '.')
Gas_df['type_conn_perc'] = Gas_df['type_conn_perc'].astype(float)
print(Gas_df['type_conn_perc'].dtype)
print(Gas_df['type_of_connection'].mode())
print(Gas_df['type_conn_perc'].median())
Gas_df['type_of_connection'] = Gas_df['type_of_connection'].fillna(Gas_df['type_of_connection'].mode()[0])
Gas_df['type_conn_perc'] = Gas_df['type_conn_perc'].fillna(Gas_df['type_conn_perc'].median())
print(Gas_df.isnull().sum().sort_values(ascending=False))

In [ ]:
Gas_df[Gas_df['smartmeter_perc'].isna()]

In [ ]:
Gas_df['smartmeter_perc'] = Gas_df['smartmeter_perc'].fillna(Gas_df['smartmeter_perc'].median())
# Gas_df['smartmeter_perc'].median()
Gas_df.describe()

In [ ]:
print(Gas_df.isnull().sum().sort_values(ascending=False))

In [ ]:
Gas_df['year'] = pd.to_datetime(Gas_df.year)

**EDA & DATA VISUALIZTION**

In [ ]:
#  We will start with visualisations for our electrical dataframe
Elec_df.head()

In [ ]:
Elec_df.info()

In [ ]:
fig,ax =  plt.subplots( 4, 2,
                       figsize = ( 15, 15))
  
sns.lineplot( x = 'year', y = "num_connections", 
             color = 'r', data = Elec_df, 
             ax = ax[0][0])
ax[0][0].tick_params(labelrotation = 45)

sns.lineplot( x = 'year', y = "delivery_perc", 
             color = 'g', data = Elec_df,
             ax = ax[0][1])
ax[0][1].tick_params(labelrotation = 45)

sns.lineplot(x = 'year', y = "perc_of_active_connections", 
             color = 'b', data = Elec_df,
             ax = ax[1][0])
ax[1][0].tick_params(labelrotation = 45)
  
sns.lineplot(x = 'year', y = "type_conn_perc", 
             color = 'r', data = Elec_df, 
             ax = ax[1][1])
ax[1][1].tick_params(labelrotation = 45)

sns.lineplot(x = 'year', y = "annual_consume", 
             color = 'g', data = Elec_df, 
             ax = ax[2][0])
ax[2][0].tick_params(labelrotation = 45)

sns.lineplot(x = 'year', y = "annual_consume_lowtarif_perc", 
             color = 'b', data = Elec_df, 
             ax = ax[2][1])
ax[2][1].tick_params(labelrotation = 45)

sns.lineplot(x = 'year', y = "smartmeter_perc", 
             color = 'r', data = Elec_df, 
             ax = ax[3][0])
ax[3][0].tick_params(labelrotation = 45)

fig.delaxes(ax[3][1])

We have been able to plot our 7 float columns and visualize their trends using a lineplot.
Some key insights can already be observed from our plots, like the drop in yearly annual consumption, the rise of smartmeter percentage, annual consumption low tarrif percentage etc

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(15,10))
# plt.figure(figsize=(15,10))
sns.countplot(x='type_of_connection', data=Elec_df, order = Elec_df['type_of_connection'].value_counts().index, ax=ax[0])
sns.barplot(x=Elec_df['year'].dt.year, y='smartmeter_perc', data=Elec_df, ax=ax[1])
ax[1].tick_params(labelrotation = 45)

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(x=Elec_df['year'].dt.year,  y='annual_consume', data=Elec_df)
plt.show()

A categorical plot of the different connection types in our dataset, we can observe a lot of largely represented connection types and some scarce connection types. And also a kde plot showing the presence of outliers in our annual consume column

In [ ]:
# Visualize Correlations using a regplot
# You can also do this using a for loop for neater code
plot_df = Elec_df.sample(frac=0.01)
fig,ax =  plt.subplots( 3, 2,
                       figsize = ( 15, 15))
  
sns.scatterplot( x = 'num_connections', y = "annual_consume", 
             color = 'r', data = plot_df, 
             ax = ax[0][0])
ax[0][0].tick_params(labelrotation = 45)

sns.scatterplot( x = 'annual_consume', y = "delivery_perc", 
             color = 'g', data = plot_df,
             ax = ax[0][1])
ax[0][1].tick_params(labelrotation = 45)

sns.scatterplot(x = 'delivery_perc', y = "annual_consume_lowtarif_perc", 
             color = 'b', data = plot_df, 
             ax = ax[1][0])
ax[1][0].tick_params(labelrotation = 45)
  
sns.scatterplot(x = 'num_connections', y = "delivery_perc", 
             color = 'r', data = plot_df,
             ax = ax[1][1])
ax[1][1].tick_params(labelrotation = 45)

sns.scatterplot(x = 'smartmeter_perc', y = "annual_consume", 
             color = 'g', data = plot_df,
             ax = ax[2][0])
ax[2][0].tick_params(labelrotation = 45)

sns.scatterplot(x = 'smartmeter_perc', y = "annual_consume_lowtarif_perc", 
             color = 'b', data = plot_df,
             ax = ax[2][1])
ax[2][1].tick_params(labelrotation = 45)

Some plots easily show your the correlations between our numerical data and others not so much, but these plots will be broken down using future plots

In [ ]:
fig,ax =  plt.subplots( 2,2,
                       figsize = ( 15, 10))

sns.regplot(x = 'num_connections', y = "smartmeter_perc", 
             color = 'r', data = plot_df, line_kws={'color': 'b'},
             ax = ax[0][0] )

sns.regplot( x = 'num_connections', y = "annual_consume", 
             color = 'g', data = plot_df, line_kws={'color': 'y'},
             ax = ax[0][1])

sns.regplot(x = 'delivery_perc', y = "annual_consume", 
             color = 'b', data = plot_df, line_kws={'color': 'g'},
             ax = ax[1][0])

sns.regplot(x = 'smartmeter_perc', y = "annual_consume_lowtarif_perc", 
             color = 'y', data = plot_df, line_kws={'color': 'b'},
             ax = ax[1][1])
plt.show()

In [ ]:
plot_df.head()
print(len(plot_df['type_of_connection'].unique()))
type_stats = plot_df.groupby('type_of_connection')['type_of_connection'].agg('count').sort_values(ascending=False)
type_stats_less = type_stats[type_stats<1000]
plot_df['type_of_connection'] = plot_df['type_of_connection'].apply(lambda x: 'other' if x in type_stats_less else x)
print(len(plot_df['type_of_connection'].unique()))
plot_df['type_of_connection'].unique()

In [ ]:
fig,ax =  plt.subplots( 2,2,
                       figsize = ( 15, 15))

sns.lineplot(x = 'num_connections', y = "annual_consume",
             data = plot_df, hue='type_of_connection', ci=None, 
             ax = ax[0][0] )
ax[0][0].tick_params(labelrotation = 45)
ax[0][0].legend(bbox_to_anchor=(-0.25, 1), loc=2, borderaxespad=0.)

sns.stripplot( x = plot_df['year'].dt.year, y='smartmeter_perc', hue='type_of_connection',
             data = plot_df,
             ax = ax[0][1])
ax[0][1].tick_params(labelrotation = 45)
ax[0][1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

sns.scatterplot(x = 'delivery_perc', y = "annual_consume", 
             color = 'b', data = plot_df, hue='type_of_connection',
             ax = ax[1][0])
ax[1][0].tick_params(labelrotation = 45)
ax[1][0].legend(bbox_to_anchor=(-0.25, 1), loc=2, borderaxespad=0.)

sns.stripplot(x = plot_df['year'].dt.year, y = "annual_consume", 
              data = plot_df, hue='type_of_connection',
             ax = ax[1][1])
ax[1][1].tick_params(labelrotation = 45)
ax[1][1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.show()

**GEOSPATIAL ANALYSIS**

We get the top ten most common cities in our dataframe and get some visualisations concerning their annual consumption, smart meter usage etc.

In [ ]:
top_10_cities = plot_df['city'].value_counts().sort_values(ascending = False).nlargest(10)
top_10_cities

In [ ]:
top_10_df = plot_df[plot_df['city'].isin(top_10_cities.index)] 
top_10_df.head()

In [ ]:
# sns.barplot(x='city', y='annual_consume', data=top_10_df)
fig, ax = plt.subplots(2,1, figsize=(15,10))
sns.barplot(x='city', y='annual_consume', data=top_10_df, ci=None, ax=ax[0])
ax[0].tick_params(rotation=45)
sns.barplot(x='city', y='smartmeter_perc', data=top_10_df, ci=None, ax=ax[1])
ax[1].tick_params(labelrotation = 45)
plt.show()

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(15,15))
sns.barplot(x=top_10_df['year'].dt.year, y='annual_consume', hue='city', data=top_10_df, ci=None, ax=ax[0])
ax[0].tick_params(rotation=45)
ax[0].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
sns.barplot(x=top_10_df['year'].dt.year, y='smartmeter_perc', hue='city', data=top_10_df, ci=None, ax=ax[1])
ax[1].tick_params(labelrotation = 45)
ax[1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
# plt.figure(figsize=(15,10))
# sns.lineplot(x = 'year', y = "annual_consume", ci=None,
#              hue='city', data = top_10_df)
# plt.xticks(rotation=35)
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
# plt.show()
fig, ax = plt.subplots(2,1, figsize=(15,15))
sns.lineplot(x=top_10_df['year'].dt.year, y='annual_consume', hue='city', data=top_10_df, ci=None, ax=ax[0])
ax[0].tick_params(rotation=45)
ax[0].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
sns.lineplot(x=top_10_df['year'].dt.year, y='delivery_perc', hue='city', data=top_10_df, ci=None, ax=ax[1])
ax[1].tick_params(labelrotation = 45)
ax[1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

We can obeserve a few things from our plots above, such as the smll drop in annual consume over the years in the most common cities. The high adoption rate of smart meters across the cities. Lastly the slow adoption of solar in some of the cities and some cities like Almere rising adoption in the green solution as seen from the drop in the delivery percentage

We try to get the cities with the highest consumption as at 2020 and see their trends over the previous years.

In [ ]:
# cities with the largest annual consumption in 2020
Elec_top_10_cities = plot_df[plot_df['year'].dt.year==2020].sort_values('annual_consume', ascending=False)['city'].head(10)
Elec_top_10_cities

In [ ]:
Elec_top_10_df = plot_df[plot_df['city'].isin(Elec_top_10_cities)] 
print(Elec_top_10_df['city'].unique())
Elec_top_10_df.head()

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(15,15))
sns.barplot(x=Elec_top_10_df['year'].dt.year, y='annual_consume', hue='city', data=Elec_top_10_df, ci=None, ax=ax[0])
ax[0].tick_params(rotation=45)
ax[0].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
sns.barplot(x=Elec_top_10_df['year'].dt.year, y='smartmeter_perc', hue='city', data=Elec_top_10_df, ci=None, ax=ax[1])
ax[1].tick_params(labelrotation = 45)
ax[1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
# plt.figure(figsize=(15,10))
# sns.lineplot(x = 'year', y = "annual_consume", ci=None,
#              hue='city', data = Elec_top_10_df)
# plt.xticks(rotation=35)
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
# plt.show()

fig, ax = plt.subplots(2,1, figsize=(15,15))
sns.lineplot(x=Elec_top_10_df['year'].dt.year, y='annual_consume', hue='city', data=Elec_top_10_df, ci=None, ax=ax[0])
ax[0].tick_params(rotation=45)
ax[0].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
sns.lineplot(x=Elec_top_10_df['year'].dt.year, y='delivery_perc', hue='city', data=Elec_top_10_df, ci=None, ax=ax[1])
ax[1].tick_params(labelrotation = 45)
ax[1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

A suprising thing to notice is the abrupt jump of annual consumtion in some of these cities, a huge spike was observed in 2020, deviating from normal, This might be due to various factors such as mass immigration, industrilizaion of the area etc. There is also a large adoption of smart meters in these cities. The solar trend of these cities all point downward with a lot of spikes, showing there might have been a need for larger energy consumption than could be provided by the solar.

We try to get the cities with the lowest delivery percentage as a result of green energy adoptions

In [ ]:
solar_top_5_cities = plot_df[plot_df['year'].dt.year==2020].sort_values('delivery_perc', ascending=True)['city'].head(5)
solar_top_5_cities

In [ ]:
solar_top_5_df = plot_df[plot_df['city'].isin(solar_top_5_cities)] 
print(solar_top_5_df['city'].unique())
solar_top_5_df.describe()

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(15,15))
sns.barplot(x=solar_top_5_df['year'].dt.year, y='annual_consume', hue='city', data=solar_top_5_df, ci=None, ax=ax[0])
ax[0].tick_params(rotation=45)
ax[0].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
sns.barplot(x=solar_top_5_df['year'].dt.year, y='smartmeter_perc', hue='city', data=solar_top_5_df, ci=None, ax=ax[1])
ax[1].tick_params(labelrotation = 45)
ax[1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(15,15))
sns.lineplot(x=solar_top_5_df['year'].dt.year, y='annual_consume', hue='city', data=solar_top_5_df, ci=None, ax=ax[0])
ax[0].tick_params(rotation=45)
ax[0].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
sns.lineplot(x=solar_top_5_df['year'].dt.year, y='delivery_perc', hue='city', data=solar_top_5_df, ci=None, ax=ax[1])
ax[1].tick_params(labelrotation = 45)
ax[1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

Most of the cities identified have their annual consumption dropping ever so lightly especially in the latter years, which can be attributed to their green energy adoption, there is also a very large acceptance for smart meters over the years and Lastly the delivery percentages are seen to decline gradually in these cities.  

I will be stopping here for my notebook but i imploy you to do the same analysis or even better analysis for the gas dataframe, also you can try building a time series model to predict annual consuption.
As always feedbcks are apppreciated and i feel i made some redundant plots. Thanks 